In [ ]:
from src.evaluator.evaluator import (
    run_subjective_evaluation,
    run_general_evaluation,
    run_rubric_evaluation,
)
from src.data import get_evaluation_dataset
import json
from pathlib import Path
from datetime import datetime

# tqdm をインポート（notebook 版が使えない場合は通常版を使用）する。
try:
    from tqdm.notebook import tqdm
except ImportError:
    from tqdm import tqdm

ITERATION_COUNT = 50

# 評価対象データを取得する。
input_data = get_evaluation_dataset()

# 評価モデルを選択する。
evaluation_model_name = "gemini-2.5-pro"

# メインループのプログレスバー。
pbar = tqdm(range(1, ITERATION_COUNT + 1), desc="全体の進捗", unit="回")

for iteration in pbar:
    pbar.set_description(f"実行中 (回 {iteration}/{ITERATION_COUNT})")
    
    # タイムスタンプディレクトリを作成する。
    timestamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
    output_dir = Path("src/data/evaluation_result") / timestamp
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # 1. Subjective Evaluation
    results = []
    for i, data in enumerate(input_data, start=1):
        # 会話履歴の構築
        conversation = ""
        for p in data['prompts']:
            conversation += f"{p['role']}: {p['content']}\n"
        conversation += f"assistant: {data['llm_response_text']}"
        
        result = run_subjective_evaluation(conversation=conversation, model_name=evaluation_model_name)
        if result:
            results.append(result)
    
    # JSON ファイルに保存する。
    if results:
        output_path = output_dir / "01_subjective_evaluation.json"
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=2)
    
    # 2. General Evaluation
    results = []
    for i, data in enumerate(input_data, start=1):
        # 会話履歴を構築する。
        conversation = ""
        for p in data['prompts']:
            conversation += f"{p['role']}: {p['content']}\n"
        conversation += f"assistant: {data['llm_response_text']}"
        
        result = run_general_evaluation(conversation=conversation, model_name=evaluation_model_name)
        if result:
            results.append(result)
    
    # JSON ファイルに保存する。
    if results:
        output_path = output_dir / "02_general_evaluation.json"
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=2)
    
    # 3. Rubric Evaluation
    results = []
    for i, data in enumerate(input_data, start=1):
        result = run_rubric_evaluation(data=data, model_name=evaluation_model_name)
        if result:
            result_dict = result.model_dump()
            results.append(result_dict)
    
    # JSON ファイルに保存する。
    if results:
        output_path = output_dir / "03_rubric_evaluation.json"
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=2)
    
    pbar.set_postfix({"完了": f"{iteration}/{ITERATION_COUNT}", "出力": str(output_dir)})

pbar.close()
print(f"\n{'#'*60}")
print(f"# 全 {ITERATION_COUNT} 回の実行が完了しました")
print(f"{'#'*60}")
